TSG046 - Knox gateway logs
==========================

Description
-----------

Knox gives a 500 error to the client, and removes details (the stack)
pointing to the cause of the underlying issue. Therefore use this TSG to
get the Knox logs from the cluster.

Steps
-----

### Parameters

In [ ]:
import re

tail_lines = 500

pod = None # All
container='knox'
log_files = [ "/var/log/knox/gateway.log" ]

expressions_to_analyze = [
    re.compile(".{23} WARN "),
    re.compile(".{23} ERROR ")
]

log_analyzer_rules = [
      [ "Invalid object name 'roles_permissions'", "TSG039", "TSG039 - Invalid object name 'role_permissions'", "../repair/tsg039-invalid-object-name-role-permissions.ipynb"],
      [ "Name node is in safe mode", "TSG024", "TSG024 - Namenode is in safe mode", "../repair/tsg024-name-node-is-in-safe-mode.ipynb"],
      [ "Connection exception dispatching request", "TSG034", "TSG034 - Livy logs", "../log-analyzers/tsg034-get-livy-logs.ipynb" ] ]

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Get tail for log

In [ ]:
# Display the last 'tail_lines' of files in 'log_files' list

pods = api.list_namespaced_pod(namespace)

entries_for_analysis = []

for p in pods.items:
    if pod is None or p.metadata.name == pod:
        for c in p.spec.containers:
            if container is None or c.name == container:
                for log_file in log_files:
                    print (f"- LOGS: '{log_file}' for CONTAINER: '{c.name}' in POD: '{p.metadata.name}'")
                    try:
                        output = stream(api.connect_get_namespaced_pod_exec, p.metadata.name, namespace, command=['/bin/sh', '-c', f'tail -n {tail_lines} {log_file}'], container=c.name, stderr=True, stdout=True)
                    except Exception:
                        print (f"FAILED to get LOGS for CONTAINER: {c.name} in POD: {p.metadata.name}")
                    else:
                        for line in output.split('\n'):
                            for expression in expressions_to_analyze:
                                if expression.match(line):
                                    entries_for_analysis.append(line)
                            print(line)
print("")
print(f"{len(entries_for_analysis)} log entries found for further analysis.")

### Analyze log entries and suggest relevant Troubleshooting Guides

In [ ]:
# Analyze log entries and suggest further relevant troubleshooting guides
from IPython.display import Markdown

print(f"Applying the following {len(log_analyzer_rules)} rules to {len(entries_for_analysis)} log entries for analysis, looking for HINTs to further troubleshooting.")
print(log_analyzer_rules)
hints = 0
if len(log_analyzer_rules) > 0:
    for entry in entries_for_analysis:
        for rule in log_analyzer_rules:
            if entry.find(rule[0]) != -1:
                print (entry)

                display(Markdown(f'HINT: Use [{rule[2]}]({rule[3]}) to resolve this issue.'))
                hints = hints + 1

print("")
print(f"{len(entries_for_analysis)} log entries analyzed (using {len(log_analyzer_rules)} rules). {hints} further troubleshooting hints made inline.")

In [ ]:
print("Notebook execution is complete.")